In [16]:
import numpy as np
import h5py
import matplotlib.pyplot as plt
import scipy.io
from Crowd_read import *

(258, 240, 320, 3)


In [19]:
m_train_index = x.shape[0]
m_train_height = x.shape[1]
m_train_weight = x.shape[2]
m_train_rgb = x.shape[3]
print(m_train_index, m_train_height, m_train_weight, m_train_rgb)
train_set_x = x. reshape(x.shape[0], -1).T
print(train_set_x.shape)

(258, 240, 320, 3)
(230400, 258)


In [20]:
def sigmoid(z):
    s = 1 / (1 + np.exp(-z))
    return s

In [8]:
def initializewithzeros(dim):
    w = np.zeros((dim, 1))
    b = 0
    
    assert (w.shape == (dim, 1))
    assert (isinstance(b, float) or isinstance(b, int))
    
    return w, b


In [21]:

def propagate(w, b, X, Y):
        
    m = X.shape[1]
    Z = np.dot(w.T, X)+b
    A = sigmoid(Z)            # compute activation
    cost = -1.0/m * np.sum(Y * np.log(A) + (1-Y) * np.log(1-A))         # compute cost
    
    dw = 1.0/m * np.dot(X, (A - Y).T)
    db = 1.0/m * np.sum(A - Y)
    
    assert(dw.shape == w.shape)
    assert(db.dtype == float)
    cost = np.squeeze(cost)
    assert(cost.shape == ())
    
    grads = {"dw": dw,
             "db": db}
    
    return grads, cost


In [22]:
def optimization(w, b, X, Y, num_iterations, learning_rate, print_cost = True):
    costs = []
    for i in range(num_iterations):
        grads, cost= propagate(w, b, X, Y)
        dw = grads["dw"]
        db = grads["db"]
        w = w - learning_rate * dw
        b = b - learning_rate * db
        if i % 100 == 0:
            costs.append(cost)
        if print_cost and i % 100 == 0:
            print ("Cost after iteration %i: %f" %(i, cost))
    
    paras = {"w" : w,
             "b" : b}
    grads = {"dw": dw,
             "db": db}
    return paras, grads, costs

In [23]:
def predict(w, b, X):
    m = X.shape[1]
    Y_prediction = np.zeros((1,m))
    w = w.reshape(X.shape[0], 1)
    A = sigmoid(np.dot(w.T, X)+b)
    for i in range(A.shape[1]):
        Y_prediction[0,i] = np.rint(A[0, i])
    assert(Y_prediction.shape == (1, m))
    return Y_prediction

In [24]:
def model(X, Y, num_iterations = 2000, learning_rate = 0.05, print_cost = True):
    w, b =initializewithzeros(X.shape[0])
    parameters, grads, costs = optimization(w, b, X, Y, num_iterations, learning_rate, print_cost)
    w = parameters["w"]
    b = parameters["b"]
    Y_prediction = predict(w, b, X)
    print(" accuracy: {} %".format(100 - np.mean(np.abs(Y_prediction - Y)) * 100))
    
    d = {"costs": costs,
         "Y_prediction_test": Y, 
         "w": w, 
         "b": b,
         "learning_rate" : learning_rate,
         "num_iterations": num_iterations}
    return d

  

In [26]:
d = model(x, Y, num_iterations=2000, learning_rate= 0.005, print_cost= True)

NameError: name 'Y' is not defined

In [27]:
costs = np.squeeze(d['costs'])
plt.plot(costs)
plt.ylabel('cost')
plt.xlabel('iterations (per hundreds)')
plt.title("Learning rate =" + str(d["learning_rate"]))
plt.show()

NameError: name 'd' is not defined